### Modelo Preditivo - Orçamento do Tratamento Hospitalar

<img src= "../imagens/Health_revenue.jpeg"  width="500" align="left">


Prever alguns valores relacionados à Internação de um Paciente é importante para a Gestão Hospitalar. Seja para Gestão da Hotelaria para administrar a estadia do paciente, seja a obtenção de informações financeiras próximas à realide para o controle e planejamento Financeiro ou de Suprimentos, existem muitas áreas que se benefeciam dessas informações.

Entretanto, pode haver pouca informação sobre a condição clínica do Paciente, exceto se houver histórico recente do mesmo ou quando o paciente tiver dado entrada na internação por meio um Atendimento de Urgência (Pronto Atendimento), uma vez que, ao solicita a internação, já haverá avaliação médica, exames, prescrição de medicamentos e outros dados valiosos.  

O Tratamento do Paciente poderá seguir diversos caminhos,  de Leitos (ocupação, capacidade e disponibilidade). Procedimentos previstos no modelo preditivo são fundamentais nos casos em que o Hospital deve oferecer orçamento quando o Convênio Médico não autoriza.

#### Gestão de Leitos
A Gestão faz o controle de ocupação e o provisionamento de Leitos Hospitalares para pacientes eletivos e ou de emergência. É um  processo complexo que envolve a admissão do paciente, a sua liberação, a limpeza e a mantenção do leito, assim como dos itens, equipamentos e toda a estrutura em perfeito estado.

In [1]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA

from scipy.stats import boxcox
from scipy.stats import shapiro
from scipy import stats

from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from prophet import Prophet
from prophet.plot import plot_plotly

from datetime import datetime, timedelta
from pandas.plotting import autocorrelation_plot

import xgboost
import statsmodels.api as sm
import plotly.offline as py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sys
import zipfile
import os

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset_path = '../datasets/'
file = 'orcamento.csv'
sys.path.append('libs')
import operacoes as op

In [5]:
#DATE_FORMAT = '%d-%m-%Y %H:%M:%S'
DATE_FORMAT = '%Y-%m-%d %H:%M:%S'

%matplotlib inline
plt.style.use('fivethirtyeight')

In [6]:
atribs_data = ['DH_ATENDIMENTO_INT','DH_ALTA_INT','DH_ATENDIMENTO_URG','DH_ALTA_URG', 'DT_NASCIMENTO']
df_hospitalization = pd.read_csv(dataset_path + file, sep=",", parse_dates=atribs_data, date_format=DATE_FORMAT)

In [7]:
df_hospitalization.head()

,Unnamed: 0,CD_PACIENTE,CD_ATENDIMENTO_INT,DH_ATENDIMENTO_INT,CD_CONVENIO_INT,DH_ALTA_INT,CD_ATENDIMENTO_URG,DH_ATENDIMENTO_URG,CD_CONVENIO_URG,DH_ALTA_URG,...,QT_HEX_SANGUE,QT_HEF_SANGUE,QT_BSF_SANGUE,QT_DIETA,DT_NASCIMENTO,TP_SEXO,TP_ESTADO_CIVIL,CD_CIDADE,IDADE,CD_DOCUMENTO
0,0,446,4824395,2023-03-29 20:41:49,7,2023-04-04 11:20:00,4823735,2023-03-29 16:30:22,7,2023-03-29 20:41:24,...,0.0,0.0,0.0,1.0,1952-06-10 02:00:00,F,C,22651,71,9.0
1,1,620,4693226,2023-02-19 01:28:07,48,2023-02-23 13:11:46,4693131,2023-02-18 21:50:10,48,2023-02-19 01:26:33,...,0.0,0.0,0.0,0.0,1951-05-20 02:00:00,F,C,22651,72,7.0
2,2,1253,5111816,2023-06-23 03:13:00,8,2023-06-26 16:22:15,5111685,2023-06-22 22:03:59,8,2023-06-23 03:10:08,...,0.0,0.0,0.0,0.0,1940-05-30 02:00:00,F,V,22651,83,9.0
3,3,1520,4401673,2022-11-14 15:09:00,5,2022-11-16 17:07:54,4399386,2022-11-14 08:07:40,5,2022-11-14 14:59:00,...,0.0,0.0,0.0,1.0,1955-04-01 02:00:00,F,C,22651,68,9.0
4,4,3067,5070049,2023-06-11 19:07:00,212,2023-06-21 05:18:52,5069796,2023-06-11 14:59:27,212,2023-06-11 19:05:03,...,0.0,0.0,0.0,0.0,1938-02-05 02:00:00,F,S,22651,85,7.0


In [8]:
df_hospitalization.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12358 entries, 0 to 12357
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          12358 non-null  int64         
 1   CD_PACIENTE         12358 non-null  int64         
 2   CD_ATENDIMENTO_INT  12358 non-null  int64         
 3   DH_ATENDIMENTO_INT  12358 non-null  datetime64[ns]
 4   CD_CONVENIO_INT     12358 non-null  int64         
 5   DH_ALTA_INT         12358 non-null  datetime64[ns]
 6   CD_ATENDIMENTO_URG  12358 non-null  int64         
 7   DH_ATENDIMENTO_URG  12358 non-null  datetime64[ns]
 8   CD_CONVENIO_URG     12358 non-null  int64         
 9   DH_ALTA_URG         12358 non-null  datetime64[ns]
 10  QT_DIAS_INTERNACAO  12358 non-null  int64         
 11  NM_CONVENIO         12358 non-null  object        
 12  TP_CONVENIO         12358 non-null  object        
 13  TOT_QT_EXA_LAB      12358 non-null  float64   

In [9]:
df_hospitalization.columns

Index(['Unnamed: 0', 'CD_PACIENTE', 'CD_ATENDIMENTO_INT', 'DH_ATENDIMENTO_INT',
       'CD_CONVENIO_INT', 'DH_ALTA_INT', 'CD_ATENDIMENTO_URG',
       'DH_ATENDIMENTO_URG', 'CD_CONVENIO_URG', 'DH_ALTA_URG',
       'QT_DIAS_INTERNACAO', 'NM_CONVENIO', 'TP_CONVENIO', 'TOT_QT_EXA_LAB',
       'TOT_VL_EXA_LAB', 'TOT_QT_EXA_IMG', 'TOT_VL_EXA_IMG',
       'VL_TOTAL_CONTA_URG', 'VL_TOTAL_CONTA_INT', 'CD_ATENDIMENTO',
       'VL_TOTAL', 'QT_CONSUMO_60', 'QT_CONSUMO_61', 'VL_CONSUMO_60',
       'VL_CONSUMO_61', 'CODIGO_ATENDIMENTO', 'QT_HEX_SANGUE', 'QT_HEF_SANGUE',
       'QT_BSF_SANGUE', 'QT_DIETA', 'DT_NASCIMENTO', 'TP_SEXO',
       'TP_ESTADO_CIVIL', 'CD_CIDADE', 'IDADE', 'CD_DOCUMENTO'],
      dtype='object')

In [15]:
numericos = ['QT_DIAS_INTERNACAO', 'TOT_QT_EXA_LAB','TOT_VL_EXA_LAB', 'TOT_QT_EXA_IMG', 'TOT_VL_EXA_IMG','VL_TOTAL_CONTA_URG', 
             'VL_TOTAL_CONTA_INT', 'VL_TOTAL', 'QT_CONSUMO_60', 'QT_CONSUMO_61', 'VL_CONSUMO_60', 'VL_CONSUMO_61', 
             'QT_HEX_SANGUE', 'QT_HEF_SANGUE', 'QT_BSF_SANGUE', 'QT_DIETA', 'IDADE']

In [12]:
op.converte_para_datetime(df_hospitalization, atribs_data, format=DATE_FORMAT)

In [16]:
df_hospitalization.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12358 entries, 0 to 12357
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          12358 non-null  int64         
 1   CD_PACIENTE         12358 non-null  int64         
 2   CD_ATENDIMENTO_INT  12358 non-null  int64         
 3   DH_ATENDIMENTO_INT  0 non-null      datetime64[ns]
 4   CD_CONVENIO_INT     12358 non-null  int64         
 5   DH_ALTA_INT         0 non-null      datetime64[ns]
 6   CD_ATENDIMENTO_URG  12358 non-null  int64         
 7   DH_ATENDIMENTO_URG  0 non-null      datetime64[ns]
 8   CD_CONVENIO_URG     12358 non-null  int64         
 9   DH_ALTA_URG         0 non-null      datetime64[ns]
 10  QT_DIAS_INTERNACAO  12358 non-null  int64         
 11  NM_CONVENIO         12358 non-null  object        
 12  TP_CONVENIO         12358 non-null  object        
 13  TOT_QT_EXA_LAB      12358 non-null  float64   

In [17]:
df_hospitalization[numericos].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12358 entries, 0 to 12357
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   QT_DIAS_INTERNACAO  12358 non-null  int64  
 1   TOT_QT_EXA_LAB      12358 non-null  float64
 2   TOT_VL_EXA_LAB      12358 non-null  float64
 3   TOT_QT_EXA_IMG      12358 non-null  float64
 4   TOT_VL_EXA_IMG      12358 non-null  float64
 5   VL_TOTAL_CONTA_URG  12358 non-null  float64
 6   VL_TOTAL_CONTA_INT  12358 non-null  float64
 7   VL_TOTAL            12358 non-null  float64
 8   QT_CONSUMO_60       12358 non-null  float64
 9   QT_CONSUMO_61       12358 non-null  float64
 10  VL_CONSUMO_60       12358 non-null  float64
 11  VL_CONSUMO_61       12358 non-null  float64
 12  QT_HEX_SANGUE       12358 non-null  float64
 13  QT_HEF_SANGUE       12358 non-null  float64
 14  QT_BSF_SANGUE       12358 non-null  float64
 15  QT_DIETA            12358 non-null  float64
 16  IDAD